# daum_news_list_final

In [1]:
import requests
import lxml.html
import pandas as pd
import sqlite3
from pandas.io import sql
import os
import time

In [9]:
def db_save(NEWS_LIST):
    with sqlite3.connect(os.path.join('.', 'sqliteDB')) as con:
        try:
            NEWS_LIST.to_sql(name = 'NEWS_LIST', con= con, index= False, if_exists ='append')
        except Exception as e:
            print(str(e))
        print(len(NEWS_LIST), '건 저장 완료...')

In [3]:
def db_delete():
    with sqlite3.connect(os.path.join('.','sqliteDB')) as con:
        try:
            cur = con.cursor()
            sql = 'DELETE FROM NEWS_LIST'
            cur.execute(sql)
        except Exception as e:
            print(str(e))

In [4]:
def db_select():
    with sqlite3.connect(os.path.join('.','sqliteDB')) as con:
        try:
            query = 'SELECT * FROM NEWS_LIST'
            NEWS_LIST = pd.read_sql(query, con=con)
        except Exception as e:
            print(str(e))
        return NEWS_LIST

In [5]:
import re
import string

def get_detail(url):
    body = []
    punc = '[!"#$%&\'()*+,-./:;<=>?[\]^_`{|}~“”·]'
    response = requests.get(url)
    root = lxml.html.fromstring(response.content)
    for p in root.xpath('//*[@id="harmonyContainer"]/section/p'):
        if p.text:
            body.append(re.sub(punc, '', p.text))
        full_body = ' '.join(body)
    
    return full_body

In [6]:
page = 58
max_page = 0
REG_DATE = '20200819'

In [14]:
while(True):
    df_list = []
    response = requests.get('http://news.daum.net/breakingnews/digital?page={}&regDate={}'\
                            .format(page, REG_DATE))
    root = lxml.html.fromstring(response.content)
    for li in root.xpath('//*[@id="mArticle"]/div[3]/ul/li'):
        a = li.xpath('div/strong/a')[0]
        url = a.get('href')
        article = get_detail(url)
        df = pd.DataFrame({'URL' : [url],
                           'TITLE' : [a.text],
                           'ARTICLE' : [article]})
        df_list.append(df)
    
    if df_list:
        df_10 = pd.concat(df_list)
        db_save(df_10)
    
    # 페이지 번호 중에서 max 페이지 가져오기
    for a in root.xpath('//*[@id="mArticle"]/div[3]/div/span/a'):
        try:
            num = int(a.text)
            if max_page < num:
                max_page = num
        except:
            pass
        
    # 마지막 페이지 여부 확인
    span = root.xpath('//*[@id="mArticle"]/div[3]/div/span/a[@class="btn_page btn_next"]')
    
    if (len(span) <= 0) & (page > max_page):
        break
    else:
        page = page + 1
    
    time.sleep(1)

13 건 저장 완료...


In [10]:
df_10

,URL,TITLE,ARTICLE
0,https://v.daum.net/v/20200819053006071,"""이태원때처럼""..기지국 정보로 광화문집회 참석자 알 수 있나",전광훈 목사가 있는 서울 성북구 사랑제일교회 관련 확진자가 5일 만에 457명대로 ...
0,https://v.daum.net/v/20200819052402971,계속되는 코로나19 돌연변이..'칵테일 항체'로 잡는다,앵커 전 세계가 코로나19 치료제와 백신 개발에 속도를 내고 있습니다 하지만 바...
0,https://v.daum.net/v/20200819050632662,"투명한 공원 공중화장실, 왜 만들었을까",공원에 설치된 공중화장실은 누구나 이용할 수 있는 개방 시설이지만 불특정 다수가 사...
0,https://v.daum.net/v/20200819050618638,부산 특구는 암호화폐를 비켜가지 않습니데이,부산광역시는 지난해 7월 국내 첫 ‘블록체인 규제자유특구’에 지정됐다 지금까지 추가...
0,https://v.daum.net/v/20200819050602624,"DAXPO 2020, 코로나19 이후 새로운 금융 토론한다",코인데스크코리아와 한국블록체인협회가 공동주최하고 부산광역시가 후원하는 디지털자산박...
0,https://v.daum.net/v/20200819042015362,코로나19 재확산 위험 커졌다.. 여름잠 준비하는 IT업계,19일 업계에 따르면 코로나19가 수도권을 중심으로 확산되면서 국내 주요 IT업체가...
0,https://v.daum.net/v/20200819030956010,"LG 벨벳, 최적의 메모리 용량 제안",고용량 메모리는 No 수년 전만 해도 보고 싶은 영화를 스마트폰에 내려받아 저장하던...
0,https://v.daum.net/v/20200819030950009,5G 보급과 함께 쑥쑥.. VR게임 대중화시대 열린다,18일 게임업계와 외신 보도 등에 따르면 올해 3월 판매를 시작한 미 게임사 밸브의...
0,https://v.daum.net/v/20200819030053787,"""게임에 빠진 아이와 소통하려면..'게임의 룰'부터 파악하세요""",방승호 서울시교육청 교육연구관59은 게임에 대한 오해가 크다고 말했다 서울 중구 충...
0,https://v.daum.net/v/20200819021530691,"""재택근무부터 온라인 콘서트까지..코로나19로 모바일이 삶에 뿌리내렸다""[신무경의 ...",글로벌 앱 분석업체 앱애니는 17일미국 현지시간 ‘신종 코로나바이러스 감염증코로나1...


In [18]:
print(db_select())

                                       URL  \
0   https://v.daum.net/v/20200819053006071   
1   https://v.daum.net/v/20200819052402971   
2   https://v.daum.net/v/20200819050632662   
3   https://v.daum.net/v/20200819050618638   
4   https://v.daum.net/v/20200819050602624   
5   https://v.daum.net/v/20200819042015362   
6   https://v.daum.net/v/20200819030956010   
7   https://v.daum.net/v/20200819030950009   
8   https://v.daum.net/v/20200819030053787   
9   https://v.daum.net/v/20200819021530691   
10  https://v.daum.net/v/20200819010010384   
11  https://v.daum.net/v/20200819004221292   
12  https://v.daum.net/v/20200819000017909   

                                                TITLE  \
0                  "이태원때처럼"..기지국 정보로 광화문집회 참석자 알 수 있나   
1                      계속되는 코로나19 돌연변이..'칵테일 항체'로 잡는다   
2                               투명한 공원 공중화장실, 왜 만들었을까   
3                             부산 특구는 암호화폐를 비켜가지 않습니데이   
4                    DAXPO 2020, 코로나19 이후 새로운 금융 토론한다   
5            

In [16]:
from string import *

In [17]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'